In [75]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#General imports - including mfcc library
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [41]:
#Get the distance between two points
def euclidean_distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [42]:
#Simple KNN function to get distance between feature vectors and find neighbors
def getKNNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = euclidean_distance(trainingset[x], instance, k) + euclidean_distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [43]:
#Nearest class
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [5]:
#Accuracy test
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [45]:
#Load data in
#Split each audio clip
#Get the MFCC for each split
#Had to remove the Jazz file - was causing errors
directory = "Data-2/genres_original/"
f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==10 :
        break  
    if directory+folder == 'Data-2/genres_original/.DS_Store':
        continue
    for file in os.listdir(directory+folder):  
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

In [13]:
#Load sets
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

In [44]:
#~75% accuracy - better than a lot of the other accuracy scores I've seen on this dataset
#Pretty good result without using CNN
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getKNNeighbors(trainingSet ,testSet[x] , 5))) 

accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

0.7573529411764706


In [67]:
#Example testing on random audio file

i=1
for folder in os.listdir("Data-2/genres_original/"):
    results[i]=folder
    i+=1

(rate,sig)=wav.read("Data-2/genres_original/disco/disco.00006.wav")
mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature=(mean_matrix,covariance,0)

pred=nearestClass(getKNNeighbors(dataset ,feature , 5))

print(results[pred])

disco
